In [1]:
# This script is meant to extract the domain information for each Camels Basin.

In [2]:
"""
Modified on Fri June 9 2020
@author: Jonathan Frame
"""

'\nModified on Fri June 5 2020\n@author: Jonathan Frame\n'

In [1]:
import datetime as dt
import geopandas as gpd 
import pandas as pd
import numpy as np
import xarray as xr
import glob
from metsim import MetSim
import netCDF4 as nc
import re

In [10]:
nl_dir = '/home/NearingLab/'
proj_dir = nl_dir + 'projects/jmframe/CAMELS_synthData/'
main_dir = proj_dir + 'nldas_forcing/'
data_dir_txt = proj_dir + 'nldas_forcing_txt/'
data_dir_nc = proj_dir + 'nldas_forcing_nc/'
domain_dir = proj_dir + 'domain_files/'
state_dir_nc = proj_dir + 'state_files/'
shapes_dir = nl_dir + 'data/camels_shapes/'

In [8]:
# import shapefile using geopandas
# Comes in with spatial reference 4269
gpd_camels = gpd.read_file(shapes_dir + 'HCDN_nhru_final_671.shp')
pd_camels = pd.DataFrame(gpd_camels)
pd_camels.set_index('hru_id', inplace=True, drop=True)

In [6]:
#Store all attributes. Really just need basin area, but the rest might come in handy.
att_path = "/home/NearingLab/data/camels_attributes_v2.0/camels_all.txt"
attributes = pd.read_csv(att_path, sep=";")
attributes.set_index('gauge_id', inplace=True)

In [9]:
def GET_LAT_LON_FROM_ATTRIBUTES(attributes, basin_id_int):
    return [attributes.loc[basin_id_int,'gauge_lat'], attributes.loc[basin_id_int,'gauge_lon']]

In [11]:
# Open up total domain file
domain_all = nc.Dataset(domain_dir + 'domain_CONUS_MX.nc')

In [29]:
domain_all

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lat(614), lon(928), month(12)
    variables(dimensions): float32 elev(lat,lon), int32 mask(lat,lon), float32 area(lat,lon), float32 frac(lat,lon), float64 lon(lon), float64 lat(lat), int64 month(month), float32 dur(month,lat,lon), float32 t_pk(month,lat,lon)
    groups: 

In [20]:
domain_lats = np.array(domain_all['lat'])
domain_lons = np.array(domain_all['lon'])

In [26]:
domain_lat_lons = {b:[] for b in attributes.index.values}
for b in attributes.index.values:
    print(b)
    lat, lon = GET_LAT_LON_FROM_ATTRIBUTES(attributes, int(b))
    min_dist_lat_lon = 999999
    for domlat in domain_lats:
        for domlon in domain_lons:
            close_lat_lon = (lat - domlat)**2 + (lon - domlon)**2
            if close_lat_lon < min_dist_lat_lon:
                min_dist_lat_lon = close_lat_lon
                domain_lat_lons[b] = [domlat, domlon]
    print(domain_lat_lons[b])

1013500
[47.21875, -68.59375]
1022500
[44.59375, -67.90625]
1030500
[45.53125, -68.28125]
1031500
[45.15625, -69.34375]
1047000
[44.84375, -69.96875]
1052500
[44.90625, -71.03125]
1054200
[44.40625, -70.96875]
1055000
[44.65625, -70.59375]
1057000
[44.28125, -70.53125]
1073000
[43.15625, -70.96875]
1078000
[43.59375, -71.71875]
1118300
[41.46875, -71.84375]
1121000
[41.84375, -72.15625]
1123000
[41.65625, -72.03125]
1134500
[44.53125, -71.84375]
1137500
[44.28125, -71.65625]
1139000
[44.15625, -72.09375]
1139800
[44.09375, -72.34375]
1142500
[43.90625, -72.65625]
1144000
[43.71875, -72.40625]
1162500
[42.65625, -72.09375]
1169000
[42.65625, -72.71875]
1170100
[42.71875, -72.65625]
1181000
[42.21875, -72.90625]
1187300
[42.03125, -72.96875]
1195100
[41.28125, -72.53125]
1333000
[42.71875, -73.21875]
1350000
[42.34375, -74.40625]
1350080
[42.40625, -74.40625]
1350140
[42.40625, -74.46875]
1365000
[41.84375, -74.46875]
1411300
[39.28125, -74.84375]
1413500
[42.15625, -74.65625]
1414500
[4

[45.28125, -84.59375]
4161580
[42.78125, -83.09375]
4185000
[41.53125, -84.40625]
4196800
[40.90625, -83.34375]
4197100
[41.03125, -83.09375]
4197170
[41.09375, -83.15625]
4213000
[41.90625, -80.59375]
4213075
[41.96875, -80.28125]
4216418
[42.84375, -78.28125]
4221000
[42.09375, -77.96875]
4224775
[42.53125, -77.71875]
4233000
[42.40625, -76.53125]
4256000
[43.71875, -75.34375]
4296000
[44.84375, -72.28125]
5056000
[47.78125, -98.71875]
5057000
[47.40625, -98.03125]
5057200
[47.21875, -98.09375]
5062500
[47.28125, -96.21875]
5087500
[48.34375, -96.84375]
5120500
[48.15625, -100.53125]
5123400
[48.59375, -100.46875]
5129115
[48.28125, -92.59375]
5131500
[48.40625, -93.53125]
5291000
[45.28125, -96.46875]
5362000
[45.28125, -90.96875]
5393500
[45.46875, -89.96875]
5399500
[44.84375, -90.09375]
5408000
[43.59375, -90.65625]
5412500
[42.71875, -91.28125]
5413500
[42.71875, -90.84375]
5414000
[42.71875, -90.65625]
5444000
[41.90625, -89.71875]
5454000
[41.71875, -91.46875]
5458000
[43.0312

[39.09375, -119.90625]
10343500
[39.40625, -120.21875]
10348850
[39.34375, -119.84375]
10396000
[42.78125, -118.84375]
11098000
[34.21875, -118.15625]
11124500
[34.59375, -119.90625]
11141280
[35.21875, -120.46875]
11143000
[36.21875, -121.78125]
11148900
[35.78125, -121.09375]
11151300
[36.28125, -121.09375]
11162500
[37.28125, -122.34375]
11176400
[37.53125, -121.65625]
11180500
[37.59375, -122.03125]
11180960
[37.71875, -122.03125]
11224500
[36.21875, -120.46875]
11230500
[37.34375, -118.96875]
11237500
[37.21875, -119.21875]
11253310
[36.40625, -120.40625]
11264500
[37.71875, -119.53125]
11266500
[37.71875, -119.65625]
11274500
[37.34375, -121.15625]
11274630
[37.46875, -121.21875]
11284400
[37.84375, -120.15625]
11299600
[37.96875, -120.59375]
11381500
[40.03125, -122.03125]
11383500
[40.03125, -121.96875]
11451100
[39.15625, -122.59375]
11468500
[39.40625, -123.71875]
11473900
[39.71875, -123.34375]
11475560
[39.71875, -123.65625]
11476600
[40.34375, -124.03125]
11478500
[40.4687

In [27]:
for b in attributes.index.values:
    print(b)
    print(GET_LAT_LON_FROM_ATTRIBUTES(attributes, int(b)))
    print(domain_lat_lons[b])

1013500
[47.23739000000001, -68.58264]
[47.21875, -68.59375]
1022500
[44.60797, -67.93524000000001]
[44.59375, -67.90625]
1030500
[45.50097, -68.30596]
[45.53125, -68.28125]
1031500
[45.17501, -69.3147]
[45.15625, -69.34375]
1047000
[44.8692, -69.9551]
[44.84375, -69.96875]
1052500
[44.87739000000001, -71.05749]
[44.90625, -71.03125]
1054200
[44.39044000000001, -70.97964]
[44.40625, -70.96875]
1055000
[44.64275, -70.58878]
[44.65625, -70.59375]
1057000
[44.30399, -70.53968]
[44.28125, -70.53125]
1073000
[43.1487, -70.96506]
[43.15625, -70.96875]
1078000
[43.56646, -71.74786]
[43.59375, -71.71875]
1118300
[41.47482, -71.83424000000002]
[41.46875, -71.84375]
1121000
[41.84371, -72.16897]
[41.84375, -72.15625]
1123000
[41.67177, -72.0523]
[41.65625, -72.03125]
1134500
[44.51172, -71.83731]
[44.53125, -71.84375]
1137500
[44.26867, -71.63036]
[44.28125, -71.65625]
1139000
[44.15034, -72.06509]
[44.15625, -72.09375]
1139800
[44.09284, -72.33565]
[44.09375, -72.34375]
1142500
[43.93451, -72.6

In [93]:
def MAKE_DOMAIN_FILES(basinID, domain_all, domain_lat_lons):
    basin_id_int = int(basinID)
    basin_id_str = "{:08}".format(basin_id_int)
    latCatchment , lonCatchment = GET_LAT_LON_FROM_ATTRIBUTES(attributes, basin_id_int)

    # Extract the data for the specific basin
    domain_prec = domain_all.variables['prec'][ : , domain_lat_lons[b][0] , domain_lat_lons[b][1] ]
    elev = domain_all.variables['elev'][ domain_lat_lons[b][0] , domain_lat_lons[b][1] ]
    mask = domain_all.variables['mask'][ domain_lat_lons[b][0] , domain_lat_lons[b][1] ]
    area = domain_all.variables['area'][ domain_lat_lons[b][0] , domain_lat_lons[b][1] ]
    frac = domain_all.variables['frac'][ domain_lat_lons[b][0] , domain_lat_lons[b][1] ]
    month = domain_all.variables['month']
    dur = domain_all.variables['dur'][ : , domain_lat_lons[b][0] , domain_lat_lons[b][1] ]
    t_pk = domain_all.variables['t_pk'][ : , domain_lat_lons[b][0] , domain_lat_lons[b][1] ]

    #BEGIN #######################################################################
    #BEGIN #  WRITE DATA TO THE NETCDF state DATA FILE   #######################
    #BEGIN #######################################################################
    
    domainDataName = domain_dir + basin_id_str + '_domain.nc'
    domain = nc.Dataset(domainDataName, 'w', format='NETCDF4_CLASSIC')
    domain.title = basin_id_str +' _domain'
    domain.description = 'Domain file for basin '+basin_id_str+' in the CAMELS catchments'

    domain.createDimension('lat', 1)
    domain.createDimension('lon', 1)
    domain.createDimension('month', size=12)

    domain.createVariable('month')
    domain.variables['month'][:] = np.arrray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

    domain.createVariable('dur', np.float32, ('month','lat','lon'))
    domain.variables['dur']_FillValue = -9999
    domain.variables['dur'].longname = "Precipitation duration (minutes)"
    domain.variables['dur'][:] = dur

    domain.createVariable('t_pk', np.float32, ('month','lat','lon'))
    domain.variables['t_pk']_FillValue = -9999
    domain.variables['t_pk'].longname = "Time of peak precipitation intensity (minutes since beginning of day)"
    domain.variables['t_pk'][:] = t_pk

    domain.createVariable('elev', np.int32, ('lat','lon'))
    domain.variables['elev']._FillValue = NaN
    domain.variables['elev'].units = 'm'
    domain.variables['elev'].long_name = 'gridcell_elevation'
    domain.variables['elev'][:] = elev

    domain.createVariable('area', np.float32, ('lat','lon'))
    domain.variables['area']._FillValue = NaN
    domain.variables['area'].units = 'm2'
    domain.variables['area'].long_name = 'area of grid cell'
    domain.variables['area'][:] = area
    domain.createVariable('mask', np.int32, ('lat','lon'))
    domain.variables['mask'].long_name = "domain mask" ;
    domain.variables['mask'].comment = "0 indicates cell is not active"

    domain.createVariable('frac', np.float32, ('lat','lon'))
    domain.variables['frac'].units = '1'
    domain.variables['frac'].longname = "fraction of grid cell that is active"
    domain.variables['frac'][:] = frac

    domain.createVariable('latitude', np.float32, ('lat',))
    domain.variables['latitude']._FillValue = NaN
    domain.variables['latitude'].units = 'degrees_north'
    domain.variables['latitude'].long_name = 'latitude'
    domain.variables['latitude'].axis = 'Y'
    domain.variables['latitude'][:] = latCatchment

    domain.createVariable('longitude', np.float32, ('lon',))
    domain.variables['longitude']._FillValue = NaN
    domain.variables['longitude'].units = 'degrees-east'
    domain.variables['longitude'].long_name = 'longitude'
    domain.variables['longitude'].axis = 'X'
    domain.variables['longitude'][:] = lonCatchment

    domain.close()

In [94]:
make_domain_files = True
if make_domain_files:
    for b in attributes.index.values:
        MAKE_DOMAIN_FILES(basinID, domain_all, domain_lat_lons)